In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bdb2024-force-data/Force_Data_1_8.parquet
/kaggle/input/force-data-1-8-all/Force_Data_1_8_total.parquet


In [2]:
def overall_tackler_stats(D):
    D['num_tackles'] = D.drop_duplicates(['gameId','playId','nflId']).query('(tackle == 1) | (assist == 1)').shape[0]
    D['total_plays'] = D.drop_duplicates(['gameId','playId','nflId']).shape[0]
    D['overal_tackle_pct'] = D.num_tackles / D.total_plays
    return D.drop(columns=['nflId'])

In [3]:
#replace this import with the file location of "Force_Data_cleaned.parquet"
force_data = pd.read_parquet('/kaggle/input/force-data-1-8-all/Force_Data_1_8_total.parquet')

In [5]:
force_data_smaller = force_data[['gameId', 'playId', 'frameId', 'nflId', 'displayName','position','last_frame', 'time',
       'jerseyNumber', 'club', 'season', 'week', 'ballCarrierId', 'ballCarrierDisplayName', 'playDescription', 'quarter',
       'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'passResult', 'expectedPoints', 'expectedPointsAdded', 
       'pff_missedTackle', 'onDefense', 'role_label','tackle','assist', 'forcedFumble','pff_missedTackle','repulse_x', 'repulse_y',
    'repulse_total', 'indiv_F_theta','repulse_y_x_ratio', 'ballc_f_x_total', 'ballc_f_y_total', 'ballc_total_F_theta', 'ballc_total_F_ratio']]

In [8]:
overall_force_data = force_data_smaller.groupby('nflId').apply(func = overall_tackler_stats).reset_index()

In [12]:
force_data_rel = overall_force_data[(overall_force_data['frameId'] >= 6) | (overall_force_data['frameId'] <= (overall_force_data['last_frame'] - 5))]

force_data_rel_completion = force_data_rel.query('passResult == "C" & onDefense == True')

In [14]:
def agg_max_func(agg_max):
    agg_max['player_avg'] = agg_max.groupby(['nflId'])['repulse_total'].transform(np.average)
    agg_max['player_sum'] = agg_max.groupby(['nflId'])['repulse_total'].transform(np.sum)
    return agg_max

In [16]:
force_data_rel_completion_agg = agg_max_func(force_data_rel_completion)

/tmp/ipykernel_42/2607593729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_max['player_avg'] = agg_max.groupby(['nflId'])['repulse_total'].transform(np.average)
/tmp/ipykernel_42/2607593729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_max['player_sum'] = agg_max.groupby(['nflId'])['repulse_total'].transform(np.sum)


In [19]:
#Defensive End Rankings
force_data_rel_completion_agg.query('total_plays >= 50').query('position == "DE"').drop_duplicates('nflId').sort_values('player_avg', ascending = False).reset_index(drop = True)

,nflId,level_1,gameId,playId,frameId,displayName,position,last_frame,time,jerseyNumber,club,season,week,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,passResult,expectedPoints,expectedPointsAdded,pff_missedTackle,onDefense,role_label,tackle,assist,forcedFumble,pff_missedTackle,repulse_x,repulse_y,repulse_total,indiv_F_theta,repulse_y_x_ratio,ballc_f_x_total,ballc_f_y_total,ballc_total_F_theta,ballc_total_F_ratio,num_tackles,total_plays,overal_tackle_pct,player_avg,player_sum
0,54491.0,753435,2022091107,301,1,Jermaine Johnson,DE,38,2022-09-11 13:14:38.799999,52.0,NYJ,2022,1,52500,Devin Duvernay,(11:19) (Shotgun) L.Jackson pass short left to...,1,2,10,BAL,NYJ,C,1.858800,0.045612,NaN,True,Defense,NaN,NaN,NaN,NaN,0.002392,0.011315,0.011565,1.362498,4.731183,0.282602,0.454515,1.014527,1.608325,10,87,0.114943,1.057067,994.700101
1,53450.0,579317,2022091105,503,1,Kwity Paye,DE,28,2022-09-11 13:23:05.299999,51.0,IND,2022,1,41246,Brandin Cooks,(5:13) (Shotgun) D.Mills pass short left to B....,1,1,10,HOU,IND,C,1.359446,-0.567517,NaN,True,Defense,NaN,NaN,NaN,NaN,0.013769,0.000479,0.013778,0.034745,0.034759,0.196427,0.137179,0.609634,0.698374,19,181,0.104972,0.891137,2054.962037
2,48462.0,857974,2022091108,1354,1,Kevin Strong,DE,25,2022-09-11 17:24:35.099999,97.0,TEN,2022,1,46071,Saquon Barkley,(5:29) D.Jones pass short middle to S.Barkley ...,2,1,10,NYG,TEN,C,1.455218,-0.315693,NaN,True,Defense,NaN,NaN,NaN,NaN,0.006823,0.016035,0.017427,1.168475,2.350000,0.207475,0.331969,1.012209,1.600043,9,97,0.092784,0.742186,898.045436
3,52641.0,922085,2022091109,163,1,Casey Toohill,DE,24,2022-09-11 13:07:49.099999,95.0,WAS,2022,1,44849,Zay Jones,(14:14) T.Lawrence pass short left to Z.Jones ...,1,1,10,JAX,WAS,C,2.950350,1.057423,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.019430,-0.001905,0.019524,0.097727,0.098039,-0.014729,-0.034164,-1.977845,2.319504,11,122,0.090164,0.689136,895.188131
4,54467.0,478741,2022091104,689,1,Aidan Hutchinson,DE,32,2022-09-11 13:31:34.700000,97.0,DET,2022,1,47834,A.J. Brown,(5:51) (Shotgun) J.Hurts pass short middle to ...,1,1,15,PHI,DET,C,-0.162443,0.912367,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.049603,-0.278109,0.282498,1.394296,5.606741,0.341108,-3.101522,-1.461256,-9.092484,13,321,0.040498,0.435041,1630.533970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,38589.0,7378828,2022101602,553,1,Vinny Curry,DE,37,2022-10-16 13:27:05.700000,99.0,NYJ,2022,6,44995,Aaron Jones,(7:48) A.Rodgers pass short right to A.Jones t...,1,1,10,GB,NYJ,C,0.966555,0.200364,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.009836,-0.097756,0.098250,1.470518,9.938775,0.168832,-0.654334,-1.318283,-3.875658,7,59,0.118644,0.027479,16.487583
89,44813.0,172851,2022091101,85,1,Myles Garrett,DE,26,2022-09-11 13:05:42.500000,95.0,CLE,2022,1,44820,Christian McCaffrey,(14:53) (Shotgun) B.Mayfield pass short left t...,1,2,10,CAR,CLE,C,0.206872,-1.836262,NaN,True,Defense,NaN,NaN,NaN,NaN,0.126598,0.196501,0.233752,0.998465,1.552163,0.402932,0.394301,0.774572,0.978579,11,247,0.044534,0.024644,84.872780
90,37841.0,5998901,2022100903,411,1,Mario Addison,DE,34,2022-10-09 13:18:37.500000,97.0,HOU,2022,5,38696,Marvin Jones,"(8:31) (No Huddle, Shotgun) T.Lawrence pass sh...",1,1,10,JAX,HOU,C,3.166933,-0.455318,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.000036,0.004444,0.004444,-1.562796,-125.000003,0.031750,-0.117166,-1.306167,-3.690253,3,91,0.032967,0.023689,31.292779
91,42436.0,190550,2022091101,662,1,Henry Anderson,DE,32,2022-09-11 13:28:22.799999,94.0,CAR,2022,1,42347,Amari Cooper,(2:58) (Shotgun) J.Brissett pass short right t...,1,1,10,CLE,CAR,C,2.984115,-0.406759,NaN,True,Defense,NaN,NaN,NaN,NaN,0.006421,-0.010033,0.011911,-1.001483,-1.562500,0.046374,0.030883,0.587510,0.665955,12,96,0.125000,0.022686,16.515110


In [20]:
#DT Rankings
force_data_rel_completion_agg.query('total_plays >= 50').query('position == "NT" | position == "DT"').drop_duplicates('nflId').sort_values('player_avg', ascending = False).reset_index(drop = True)

,nflId,level_1,gameId,playId,frameId,displayName,position,last_frame,time,jerseyNumber,club,season,week,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,passResult,expectedPoints,expectedPointsAdded,pff_missedTackle,onDefense,role_label,tackle,assist,forcedFumble,pff_missedTackle,repulse_x,repulse_y,repulse_total,indiv_F_theta,repulse_y_x_ratio,ballc_f_x_total,ballc_f_y_total,ballc_total_F_theta,ballc_total_F_ratio,num_tackles,total_plays,overal_tackle_pct,player_avg,player_sum
0,47841.0,1250357,2022091113,465,1,Trysten Hill,DT,36,2022-09-11 20:37:58.700000,72.0,DAL,2022,1,41233,Mike Evans,(7:58) T.Brady pass deep right to M.Evans to D...,1,2,8,TB,DAL,C,2.281414,1.738313,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.001377,0.004147,0.004370,-1.250095,-3.010526,-0.214407,0.326294,2.152145,-1.521845,7,105,0.066667,2.304657,2668.793036
1,53073.0,835657,2022091108,318,1,Teair Tart,DT,28,2022-09-11 16:39:34.700000,93.0,TEN,2022,1,46071,Saquon Barkley,(9:44) (Shotgun) D.Jones pass short left to S....,1,2,10,NYG,TEN,C,0.786975,-0.370533,NaN,True,Defense,NaN,NaN,NaN,NaN,0.001685,0.014401,0.014500,1.454324,8.546875,0.210364,0.149906,0.619136,0.712605,14,168,0.083333,1.388450,3011.548566
2,52422.0,274963,2022091102,364,1,Javon Kinlaw,DT,60,2022-09-11 13:15:51.799999,99.0,SF,2022,1,47856,David Montgomery,(9:10) (Shotgun) J.Fields pass short middle to...,1,3,6,CHI,SF,C,-0.763683,2.563382,1.0,True,Defense,0.0,0.0,0.0,1.0,0.024065,-0.177924,0.179544,-1.436357,-7.393443,0.091947,-0.411639,-1.351035,-4.476896,1,52,0.019231,1.075614,578.680249
3,54615.0,2351300,2022091810,1400,1,Thomas Booker,DT,27,2022-09-18 17:16:41.400000,56.0,HOU,2022,2,42358,Melvin Gordon,(13:31) (Shotgun) R.Wilson pass short middle t...,2,3,16,DEN,HOU,C,0.367298,-0.876291,NaN,True,Defense,NaN,NaN,NaN,NaN,0.032182,0.063683,0.071353,1.102880,1.978836,0.360557,0.315626,0.719047,0.875384,8,84,0.095238,0.944785,594.269472
4,47867.0,1020884,2022091110,870,1,Khalen Saunders,DT,27,2022-09-11 16:53:49.299999,99.0,KC,2022,1,48646,Greg Dortch,(:04) (Shotgun) K.Murray pass deep right to G....,1,2,18,ARI,KC,C,1.183888,2.611672,NaN,True,Defense,NaN,NaN,NaN,NaN,0.006540,-0.004215,0.007781,-0.572527,-0.644540,0.046623,-0.132666,-1.232847,-2.845508,26,127,0.204724,0.732159,1134.113694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,42360.0,282314,2022091102,756,1,Arik Armstead,DT,27,2022-09-11 13:35:10.400000,91.0,SF,2022,1,53646,Khalil Herbert,(3:55) J.Fields pass short left to K.Herbert t...,1,1,10,CHI,SF,C,1.530588,-1.123506,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.017923,0.035640,0.039893,-1.104839,-1.988506,0.236297,0.128022,0.496512,0.541782,4,51,0.078431,0.027697,14.125549
122,53621.0,1248497,2022091113,413,1,Quinton Bohanna,DT,50,2022-09-11 20:36:42.000000,98.0,DAL,2022,1,44896,Chris Godwin,(9:02) (Shotgun) T.Brady pass short left to C....,1,1,15,TB,DAL,C,0.500792,2.198878,NaN,True,Defense,NaN,NaN,NaN,NaN,0.023003,0.004700,0.023478,0.201528,0.204301,0.196640,-0.088875,-0.424489,-0.451968,9,127,0.070866,0.025472,31.915790
123,38619.0,1238181,2022091113,65,1,Akiem Hicks,DT,22,2022-09-11 20:23:43.299999,96.0,TB,2022,1,46206,Dalton Schultz,(14:56) (Shotgun) D.Prescott pass short left t...,1,1,10,DAL,TB,C,0.444005,0.224793,NaN,True,Defense,NaN,NaN,NaN,NaN,0.015828,0.011750,0.019713,0.638575,0.742331,1.508966,-0.527786,-0.336467,-0.349766,5,58,0.086207,0.020309,11.677571
124,44922.0,10165255,2022103004,870,1,Jaleel Johnson,DT,45,2022-10-30 16:50:16.000000,90.0,HOU,2022,8,43334,Derrick Henry,(13:31) M.Willis pass short right to D.Henry t...,2,1,10,TEN,HOU,C,0.676076,0.682049,NaN,True,Defense,NaN,NaN,NaN,NaN,0.000466,-0.007224,0.007239,-1.506418,-15.511628,0.097223,0.119416,0.887486,1.228273,3,53,0.056604,0.018070,7.011116


In [21]:
#Safety Rankings
force_data_rel_completion_agg.query('total_plays >= 50').query('position == "SS" | position == "FS"').drop_duplicates('nflId').sort_values('player_avg', ascending = False).reset_index(drop = True)

,nflId,level_1,gameId,playId,frameId,displayName,position,last_frame,time,jerseyNumber,club,season,week,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,passResult,expectedPoints,expectedPointsAdded,pff_missedTackle,onDefense,role_label,tackle,assist,forcedFumble,pff_missedTackle,repulse_x,repulse_y,repulse_total,indiv_F_theta,repulse_y_x_ratio,ballc_f_x_total,ballc_f_y_total,ballc_total_F_theta,ballc_total_F_ratio,num_tackles,total_plays,overal_tackle_pct,player_avg,player_sum
0,41256.0,9366763,2022102310,335,1,Jimmie Ward,FS,31,2022-10-23 16:35:56.599999,1.0,SF,2022,7,47839,Mecole Hardman,(9:43) (Shotgun) P.Mahomes pass short left to ...,1,2,8,KC,SF,C,0.815707,-0.763247,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.008569,-0.034275,0.035330,1.325818,4.000000,0.003732,0.035564,1.466243,9.529583,7,76,0.092105,4.277116,5483.263169
1,47964.0,2657127,2022091900,3024,1,Jaquan Johnson,FS,32,2022-09-19 21:47:20.700000,4.0,BUF,2022,2,43334,Derrick Henry,"(4:24) (No Huddle, Shotgun) R.Tannehill pass s...",3,2,2,TEN,BUF,C,2.867526,-1.479203,NaN,True,Defense,NaN,NaN,NaN,NaN,0.000306,0.001531,0.001561,1.373401,5.000000,-0.025250,0.016454,2.564068,-0.651637,24,153,0.156863,2.826659,5947.289808
2,46097.0,376924,2022091103,295,1,Terrell Edmunds,SS,23,2022-09-11 13:16:13.400000,34.0,PIT,2022,1,43344,Tyler Boyd,"(11:34) (No Huddle, Shotgun) J.Burrow pass sho...",1,1,10,CIN,PIT,C,2.435449,-0.338776,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.010355,-0.005105,0.011545,0.458066,0.493042,0.050189,-0.156302,-1.260095,-3.114281,31,319,0.097179,2.351888,9012.435711
3,48581.0,1133296,2022091111,1975,1,Roderic Teamer,SS,19,2022-09-11 18:04:40.900000,33.0,LV,2022,1,53526,Tre' McKitty,(11:17) (Shotgun) J.Herbert pass short middle ...,3,1,10,LAC,LV,C,2.331285,0.768697,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.000099,-0.007428,0.007429,1.557428,74.799999,0.004771,0.046708,1.469004,9.789999,9,52,0.173077,2.325530,1648.800902
4,52613.0,2762663,2022091901,3002,1,Josh Metellus,SS,69,2022-09-19 22:39:58.500000,44.0,MIN,2022,2,53579,Kenneth Gainwell,(12:41) (Shotgun) J.Hurts pass short left to K...,4,3,14,PHI,MIN,C,-0.081736,-0.313208,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.001333,0.001423,0.001949,-0.818114,-1.067669,0.054393,0.070899,0.916382,1.303452,13,68,0.191176,1.886554,1533.768244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,41270.0,749344,2022091107,202,1,Lamarcus Joyner,SS,32,2022-09-11 13:12:14.700000,29.0,NYJ,2022,1,53456,Rashod Bateman,(13:42) L.Jackson pass short right to R.Batema...,1,1,10,BAL,NYJ,C,1.386474,0.062406,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.000158,-0.002224,0.002230,1.500048,14.111110,-0.019958,-0.124379,-1.729900,6.232077,34,420,0.080952,0.493746,2547.728100
99,54704.0,3105570,2022092502,1325,1,Rodney Thomas,FS,23,2022-09-25 14:02:13.799999,25.0,IND,2022,3,44874,JuJu Smith-Schuster,(6:24) (Shotgun) P.Mahomes pass short left to ...,2,2,6,KC,IND,C,2.103809,0.710932,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.002932,-0.003095,0.004263,0.812419,1.055556,0.217075,-0.029685,-0.135906,-0.136749,23,246,0.093496,0.466189,1365.933641
100,48026.0,352,2022090800,56,1,Nick Scott,SS,22,2022-09-08 20:24:05.200000,33.0,LA,2022,1,42489,Stefon Diggs,(15:00) (Shotgun) J.Allen pass short right to ...,1,1,10,BUF,LA,C,1.298699,0.004420,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.004284,0.010746,0.011569,-1.191436,-2.508333,0.202956,0.451391,1.148256,2.224084,34,346,0.098266,0.462808,2284.881656
101,42543.0,1328037,2022091200,286,1,Quandre Diggs,FS,56,2022-09-12 20:25:15.299999,6.0,SEA,2022,1,48096,Andrew Beck,(11:18) R.Wilson pass short right to A.Beck to...,1,2,10,DEN,SEA,C,-0.279236,2.353510,NaN,True,Defense,NaN,NaN,NaN,NaN,0.017234,-0.006763,0.018513,-0.373942,-0.392405,0.158852,-0.245894,-0.997224,-1.547940,39,428,0.091121,0.401321,2206.061396


In [22]:
#Linebacker Rankings
force_data_rel_completion_agg.query('total_plays >= 50').query('position == "OLB" | position == "MLB" | position == "ILB"').drop_duplicates('nflId').sort_values('player_avg', ascending = False).reset_index(drop = True)

,nflId,level_1,gameId,playId,frameId,displayName,position,last_frame,time,jerseyNumber,club,season,week,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,passResult,expectedPoints,expectedPointsAdded,pff_missedTackle,onDefense,role_label,tackle,assist,forcedFumble,pff_missedTackle,repulse_x,repulse_y,repulse_total,indiv_F_theta,repulse_y_x_ratio,ballc_f_x_total,ballc_f_y_total,ballc_total_F_theta,ballc_total_F_ratio,num_tackles,total_plays,overal_tackle_pct,player_avg,player_sum
0,43325.0,376694,2022091103,295,1,Myles Jack,ILB,23,2022-09-11 13:16:13.400000,51.0,PIT,2022,1,43344,Tyler Boyd,"(11:34) (No Huddle, Shotgun) J.Burrow pass sho...",1,1,10,CIN,PIT,C,2.435449,-0.338776,0.0,True,Defense,0.0,1.0,0.0,0.0,0.068416,0.047183,0.083108,0.603749,0.689655,0.050189,-0.156302,-1.260095,-3.114281,67,340,0.197059,3.947524,16824.347839
1,46175.0,4024531,2022092513,86,1,Josey Jewell,ILB,51,2022-09-25 20:24:03.700000,47.0,DEN,2022,3,47819,Deebo Samuel,(14:19) J.Garoppolo pass short left to D.Samue...,1,2,8,SF,DEN,C,-0.329895,1.663811,NaN,True,Defense,NaN,NaN,NaN,NaN,0.013806,-0.026435,0.029823,-1.089489,-1.914706,0.111103,-0.128589,-0.858220,-1.157382,35,164,0.213415,3.898201,6892.019089
2,43420.0,7034702,2022101000,1427,1,Blake Martinez,ILB,13,2022-10-10 21:18:33.400000,54.0,LV,2022,5,46243,Marquez Valdes-Scantling,(6:15) (Shotgun) P.Mahomes pass short middle t...,2,1,10,KC,LV,C,4.292064,0.859268,NaN,True,Defense,NaN,NaN,NaN,NaN,0.000208,-0.027095,0.027096,-1.563104,-129.999998,-0.001400,-0.292083,-1.575589,208.649746,20,93,0.215054,3.796134,3879.649197
3,47855.0,372664,2022091103,58,1,Germaine Pratt,OLB,21,2022-09-11 13:04:14.200000,57.0,CIN,2022,1,53484,Pat Freiermuth,(15:00) (Shotgun) M.Trubisky pass short middle...,1,1,10,PIT,CIN,C,1.298699,-0.472126,NaN,True,Defense,NaN,NaN,NaN,NaN,0.012707,0.027307,0.030119,1.135254,2.148936,0.752586,0.898658,0.873630,1.194093,57,274,0.208029,3.706324,7534.956308
4,54523.0,2115968,2022091807,839,1,Troy Andersen,ILB,32,2022-09-18 16:40:54.000000,44.0,ATL,2022,2,43399,Tyler Higbee,(14:11) (Shotgun) M.Stafford pass short right ...,2,1,10,LA,ATL,C,2.771719,0.152558,0.0,True,Defense,0.0,1.0,0.0,0.0,0.012365,0.064490,0.065664,1.381366,5.215687,0.592860,0.441987,0.640625,0.745516,27,127,0.212598,3.459788,8061.305955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,37145.0,755800,2022091107,412,1,Justin Houston,OLB,20,2022-09-11 13:20:37.299999,50.0,BAL,2022,1,54501,Breece Hall,(9:11) (Shotgun) J.Flacco pass short left to B...,1,2,12,NYJ,BAL,C,-0.630774,-0.199966,NaN,True,Defense,NaN,NaN,NaN,NaN,0.005194,-0.001192,0.005329,-0.225601,-0.229508,0.315017,-0.129071,-0.388865,-0.409729,4,111,0.036036,0.051426,125.634451
162,42465.0,1175520,2022091112,611,1,Za'Darius Smith,OLB,16,2022-09-11 16:52:40.700000,55.0,MIN,2022,1,37139,Randall Cobb,(4:03) (Shotgun) A.Rodgers pass short left to ...,1,3,4,GB,MIN,C,1.548537,1.566405,NaN,True,Defense,NaN,NaN,NaN,NaN,0.022233,-0.005440,0.022889,-0.239966,-0.244681,0.764444,-0.760908,-0.783080,-0.995374,15,270,0.055556,0.043011,182.195084
163,47795.0,1169452,2022091112,62,1,Rashan Gary,OLB,34,2022-09-11 16:26:32.599999,52.0,GB,2022,1,45094,Johnny Mundt,(14:54) K.Cousins pass short left to J.Mundt p...,1,1,10,MIN,GB,C,0.906152,0.794783,NaN,True,Defense,NaN,NaN,NaN,NaN,0.230275,0.195094,0.301808,0.702879,0.847222,0.161010,0.437870,1.218430,2.719518,24,269,0.089219,0.037081,105.197914
164,43292.0,1098728,2022091111,336,1,Joey Bosa,OLB,22,2022-09-11 16:35:47.599999,97.0,LAC,2022,1,41282,Davante Adams,(9:03) (Shotgun) D.Carr pass short right to D....,1,1,10,LV,LAC,C,1.381075,0.730491,NaN,True,Defense,NaN,NaN,NaN,NaN,0.001027,-0.024961,0.024982,-1.529694,-24.315789,0.241652,-0.174782,-0.626178,-0.723278,4,76,0.052632,0.025856,30.277271


In [23]:
#Defensive Back Rankings
force_data_rel_completion_agg.query('total_plays >= 50').query('position == "CB" | position == "DB"').drop_duplicates('nflId').sort_values('player_avg', ascending = False).reset_index(drop = True)

,nflId,level_1,gameId,playId,frameId,displayName,position,last_frame,time,jerseyNumber,club,season,week,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,passResult,expectedPoints,expectedPointsAdded,pff_missedTackle,onDefense,role_label,tackle,assist,forcedFumble,pff_missedTackle,repulse_x,repulse_y,repulse_total,indiv_F_theta,repulse_y_x_ratio,ballc_f_x_total,ballc_f_y_total,ballc_total_F_theta,ballc_total_F_ratio,num_tackles,total_plays,overal_tackle_pct,player_avg,player_sum
0,41483.0,3524645,2022092507,501,1,Terrance Mitchell,CB,19,2022-09-25 13:19:43.200000,39.0,TEN,2022,3,47807,Josh Jacobs,(5:53) (Shotgun) D.Carr pass short left to J.J...,1,2,14,LV,TEN,C,2.566176,-1.315406,NaN,True,Defense,NaN,NaN,NaN,NaN,0.000290,0.011271,0.011274,1.545051,38.833333,0.606485,0.987888,1.020204,1.628875,23,137,0.167883,4.470590,10930.593075
1,54622.0,6543433,2022100908,3537,1,Zyon McCollum,CB,31,2022-10-09 15:43:13.299999,27.0,TB,2022,5,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,C,2.360609,0.981955,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.005439,-0.004167,0.006852,0.653744,0.766129,0.211318,0.330396,1.001774,1.563502,10,108,0.092593,3.734241,4932.932870
2,54500.0,835713,2022091108,318,1,Roger McCreary,CB,28,2022-09-11 16:39:34.700000,21.0,TEN,2022,1,46071,Saquon Barkley,(9:44) (Shotgun) D.Jones pass short left to S....,1,2,10,NYG,TEN,C,0.786975,-0.370533,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.001779,0.003622,0.004035,-1.114317,-2.036364,0.210364,0.149906,0.619136,0.712605,50,366,0.136612,3.585570,20552.486337
3,54606.0,827861,2022091107,3811,1,Damarion Williams,CB,35,2022-09-11 15:54:07.200000,22.0,BAL,2022,1,54501,Breece Hall,(2:37) (Shotgun) J.Flacco pass short right to ...,4,2,10,NYJ,BAL,C,2.454007,1.335166,NaN,True,Defense,NaN,NaN,NaN,NaN,0.023932,-0.015964,0.028767,-0.588268,-0.667051,0.295440,-0.387892,-0.919877,-1.312929,20,129,0.155039,2.612328,6494.247529
4,46775.0,1411412,2022091500,118,1,J.C. Jackson,CB,42,2022-09-15 20:18:16.099999,27.0,LAC,2022,2,52440,Clyde Edwards-Helaire,(13:20) (Shotgun) P.Mahomes pass short left to...,1,1,10,KC,LAC,C,2.368279,0.654809,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.041102,0.028365,0.049940,-0.604054,-0.690104,0.016959,0.653804,1.544863,38.551112,14,167,0.083832,2.397866,5553.458002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,42393.0,1324925,2022091200,109,1,Ronald Darby,CB,25,2022-09-12 20:17:33.799999,23.0,DEN,2022,1,42412,Tyler Lockett,(13:38) (Shotgun) G.Smith pass short middle to...,1,3,3,SEA,DEN,C,0.950914,2.206573,NaN,True,Defense,NaN,NaN,NaN,NaN,0.003176,-0.015796,0.016112,-1.372370,-4.973333,0.254466,-0.107531,-0.399813,-0.422573,14,190,0.073684,0.378753,882.493476
133,52438.0,6369017,2022100907,295,1,Noah Igbinoghene,CB,23,2022-10-09 13:14:07.000000,9.0,MIA,2022,5,46279,Braxton Berrios,(11:14) (Shotgun) Z.Wilson pass short left to ...,1,2,9,NYJ,MIA,C,2.968859,-0.885419,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.002387,0.011982,0.012218,-1.374167,-5.020000,0.334892,0.249864,0.641003,0.746103,9,129,0.069767,0.353022,673.566834
134,46215.0,427249,2022091103,2569,1,Tre Flowers,CB,25,2022-09-11 15:07:01.400000,33.0,CIN,2022,1,47924,Zach Gentry,(1:00) (Shotgun) M.Trubisky pass short middle ...,3,3,16,PIT,CIN,C,-0.629734,-0.366711,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.002008,-0.010127,0.010324,1.375096,5.044445,0.217886,-0.443228,-1.113900,-2.034221,13,63,0.206349,0.332853,410.741010
135,52458.0,268434,2022091102,189,1,Jaylon Johnson,CB,48,2022-09-11 13:08:54.000000,33.0,CHI,2022,1,47819,Deebo Samuel,(12:33) (Shotgun) T.Lance pass short middle to...,1,2,8,SF,CHI,C,2.270418,-0.130476,NaN,True,Defense,NaN,NaN,NaN,NaN,-0.029872,-0.108626,0.112658,1.302430,3.636364,0.131178,-1.840713,-1.499652,-14.032137,14,256,0.054688,0.287685,820.765920


In [25]:
force_leaderboard_data = force_data_rel_completion_agg.drop_duplicates('nflId')[['nflId','displayName','position','club','num_tackles','total_plays','repulse_total','player_avg','overal_tackle_pct','player_sum']]

In [26]:
force_leaderboard_send_data.to_csv('force_leaderboard_full_data.csv')

In [ ]:
#Check correlation of TRU with Overall Tackle Percentage

In [28]:
force_data_rel_completion_agg['overal_tackle_pct'].corr(force_data_rel_completion_agg['player_avg'])

0.5161062339268756

In [ ]:
#Check correlation of TRU for first 5 weeks with last 4 weeks

In [29]:
#Get aggregated TRU data for first 5 weeks
overall_force_data_first_5 = force_data_smaller.query('week < 6 & onDefense == True').groupby('nflId').apply(func = overall_tackler_stats).reset_index()

overall_force_data_first_5_agg = agg_max_func(overall_force_data_first_5)

In [31]:
#Get aggregated TRU data for last 4 weeks
overall_force_data_last_4 = force_data_smaller.query('week >= 6 & onDefense == True').groupby('nflId').apply(func = overall_tackler_stats).reset_index()

overall_force_data_last_4_agg = agg_max_func(overall_force_data_last_4)

In [33]:
overall_force_data_first_5_agg = overall_force_data_first_5_agg.drop_duplicates('nflId')[['nflId','displayName','position','club','num_tackles','total_plays','overal_tackle_pct', 'player_avg','player_sum']]
overall_force_data_last_4_agg = overall_force_data_last_4_agg.drop_duplicates('nflId')[['nflId','displayName','position','club','num_tackles','total_plays','overal_tackle_pct', 'repulse_total', 'player_avg','player_sum']]

overall_force_data_first_5_agg = overall_force_data_first_5_agg.rename(columns = {'total_plays': 'total_plays_first','overal_tackle_pct': 'overall_tackle_pct_first', 'player_avg': 'player_avg_first','player_sum': 'player_sum_first'})

overall_force_data_last_4_agg = overall_force_data_last_4_agg.rename(columns = {'total_plays': 'total_plays_second','overal_tackle_pct': 'overall_tackle_pct_second', 'player_avg': 'player_avg_second','player_sum': 'player_sum_second'})

In [38]:
#Check correlation of first 5 weeks with last 4 weeks
merged_data = pd.merge(overall_force_data_first_5_agg, overall_force_data_last_4_agg, on='nflId', how='inner')

In [44]:
data_halves_corr = merged_data[['nflId','displayName_x','position_x','club_x','total_plays_first','overall_tackle_pct_first','player_avg_first','player_sum_first','total_plays_second','overall_tackle_pct_second','player_avg_second','player_sum_second']]

In [40]:
merged_data_filter = merged_data.query('total_plays_first >= 30 & total_plays_second >= 30')

In [41]:
#Correlation for first half with second half of data
merged_data_filter['player_avg_first'].corr(merged_data_filter['player_avg_second'])

0.2900456982241231

In [46]:
data_halves_corr.query('total_plays_first >= 30 & total_plays_second >= 30')['player_avg_first'].corr(data_halves_corr.query('total_plays_first >= 30 & total_plays_second >= 30')['player_avg_second'])

0.2900456982241231

In [47]:
data_halves_corr.to_csv('data_halves_corr.csv')